<div style="display: flex; justify-content: space-between; align-items: center; margin-bottom: 40px; margin-top: 0;">
    <div style="flex: 0 0 auto; margin-left: 0; margin-bottom: 0; margin-top: 0;">
        <img src="./pics/UCSD Logo.png" alt="UCSD Logo" style="width: 179px; margin-bottom: 0px; margin-top: 20px;">
    </div>
    <div style="flex: 0 0 auto; margin-left: auto; margin-bottom: 0; margin-top: 20px;">
        <img src="./pics/silvxlabs.png" alt="SilvX Labs Logo" style="width: 200px; margin-bottom: 0px;">
    </div>
    <div style="flex: 0 0 auto; margin-left: auto; margin-bottom: 0; margin-top: 20px;">
        <img src="./pics/sdsclogo-plusname-horiz-red.jpg" alt="San Diego Supercomputer Center Logo" width="300"/>
    </div>
</div>

<h1 style="text-align: center; font-size: 48px; margin-top: 0;">6NRP Demo Data Challenge</h1>

### Acknowledgment

This Jupyter Notebook was authored by Anthony Marcozzi from the New Mexico Consortium. The content, including the code, analysis, and visualizations, reflects the author's expertise and effort in creating an accessible and insightful resource. Proper credit is appreciated if this notebook is shared or adapted for further use.

# Base Solution

In this notebook we will train simple, univariate random forest models to predict the diameter, species, and crown base height of a tree using only an observation on the tree's height. For training data we'll utilize the California daatabase from the Forest Inventory and Analysis (FIA) database. 

Random forests are a straightforward modeling approach based on voting from a large number of decision trees. In this notebook we will use a [prebuilt model from scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) rather than programming our own. 
There's no background knolwedge on what random forest models are or how they work required, but there are a lot of good resources out there for learning more. 
Some recommended resources are listed below:

1) https://builtin.com/data-science/random-forest-algorithm
2) https://developers.google.com/machine-learning/decision-forests/random-forests

In [ ]:
import folium
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from pathlib import Path
import matplotlib.colors as mcolors
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

In [ ]:
DATA_PATH = Path("data")

In [ ]:
# Load the detected trees from the ALS data into a geopandas dataframe
lidar_trees = pd.read_csv(DATA_PATH / 'ttops.csv')
lidar_trees = gpd.GeoDataFrame(lidar_trees, geometry=gpd.points_from_xy(lidar_trees.y, lidar_trees.x), crs='EPSG:4326')

# Load the FIA data
fia_ca_plot_table = pd.read_csv(DATA_PATH / 'fia' / 'CA_PLOT.csv', low_memory=False)
fia_ca_tree_table = pd.read_csv(DATA_PATH / 'fia' / 'CA_TREE.csv', low_memory=False)
fia_ref_species_table = pd.read_csv(DATA_PATH / 'fia' / 'REF_SPECIES.csv')

# Filter the PLOT table for the ECOSUBCD
fia_ca_plot_table = fia_ca_plot_table[fia_ca_plot_table['ECOSUBCD'] == "M261Ej"]

# Merge the PLOT and TREE tables
trees_fia = pd.merge(fia_ca_tree_table, fia_ca_plot_table, left_on='PLT_CN', right_on='CN')

# Create a dictionary mapping SPCD to COMMON_NAME
spcd_to_common_name = dict(zip(fia_ref_species_table['SPCD'], fia_ref_species_table['COMMON_NAME']))


## Predicting Tree Variables

Next, let's use the FIA data as training data for our random forest model. 
We will train a different model for each variable that we are interested in predicting. 
This means that we will have a model to predict diameter from height, a second model to predict species from height, and a third model to predict crown base height from total height.

Student implementations can look at different approaches including models that predict multiple variables. 

## Diameter from height

Our first random forest model will take in height (HT) as input and produce diameter (DIA) as output.

In [ ]:
# Split into training and test sets
independent_variables = ["HT"]
dependent_variable = "DIA"
include_variables = independent_variables + [dependent_variable]
trees_train, trees_test = train_test_split(trees_fia[include_variables].dropna(), test_size=0.2)
print(f"Training set size: {len(trees_train)}")
print(f"Test set size: {len(trees_test)}")

In [ ]:
# Train the model
model_ht = RandomForestRegressor()
model_ht.fit(trees_train[independent_variables], trees_train[dependent_variable])

# Compute R^2 and RMSE using the test set
print(f"Model R^2: {model_ht.score(trees_test[independent_variables], trees_test[dependent_variable]):.2f}")
print(f"Model RMSE: {((model_ht.predict(trees_test[independent_variables]) - trees_test[dependent_variable])**2).mean()**0.5:.2f} inches")


# Compare the predicted diameter to the actual diameter.
fig, ax = plt.subplots()
trees_test["predicted_diameter"] = model_ht.predict(trees_test[independent_variables])
trees_test.plot.scatter(x="DIA", y="predicted_diameter", ax=ax)
upper_dia_limit = max(trees_test["DIA"].max(), trees_test["predicted_diameter"].max()) + 1
ax.plot([0, upper_dia_limit], [0, upper_dia_limit], color='red')
ax.set_xlabel("Actual Diameter (in)")
ax.set_ylabel("Predicted Diameter (in)")
ax.set_title("Validation Data Diameter Prediction")
plt.show()

That's pretty good for a simple model! Accuracy decreases as diameter increases, but up to around 30 inches it is quite accurate. It may be worth investigating approaches for outlier detection and removal. For example, maybe we can improve model performance by only considering diameter estimates in a certain range. Consider investigating the diameter range of the validation data to further constrain the problem.

Next, let's use our new model to predict the diameters using the heights we observed from the ALS LIDAR data.

In [ ]:
# User our new model to predict the diameter of the trees detected in the ALS data
lidar_trees["predicted_diameter"] = model_ht.predict(lidar_trees[independent_variables])

In [ ]:
# Plot the predicted diameter
fig, ax = plt.subplots()
lidar_trees.plot.scatter(x="HT", y="predicted_diameter", ax=ax)
ax.set_xlabel("Height (ft)")
ax.set_ylabel("Predicted Diameter (in)")
ax.set_title("Independence Lake Tree Diameter Prediction")
plt.show()

### SPCD (Species code) from height

Species code, or SPCD, is a numeric identifier for tree species across the United States. Tree species is an enormously important characteristic for making predictions about tree biomass, carbon content, size, and more. Unfortunately, we don't learn tree species from the ALS acquistion data so we want to try and predict it using a model trained on FIA data. In this example, we train a simple random forest classifer to predict trees species based just on the height of the tree. Let's see how well this simple model performs.

In [ ]:
# Split into training and test sets
independent_variables = ["HT"]
dependent_variable = "SPCD"
include_variables = independent_variables + [dependent_variable]
trees_train, trees_test = train_test_split(trees_fia[include_variables].dropna(), test_size=0.2)
print(f"Training set size: {len(trees_train)}")
print(f"Test set size: {len(trees_test)}")

# Train the model
model_spcd = RandomForestClassifier()
model_spcd.fit(trees_train[independent_variables], trees_train[dependent_variable])

# Evaluate classification accuracy
unique_species = sorted(trees_test[dependent_variable].unique())
species_names = [spcd_to_common_name.get(spcd, f"Unknown ({spcd})") for spcd in unique_species]
report = classification_report(trees_test[dependent_variable], model_spcd.predict(trees_test[independent_variables]), zero_division=0, target_names=species_names)
print(report)

So how did we do? We performed slightly better than taking a random guess across the four most common species, but with an f1-score of 0.35 there is a lot of room for improvement.

Let's take a look at the confusion matrix and see if there are any obvious errors.


In [ ]:
cm = confusion_matrix(trees_test[dependent_variable], model_spcd.predict(trees_test[independent_variables]))
display = ConfusionMatrixDisplay(cm, display_labels=species_names)
display.plot(cmap='Blues', values_format='d')
plt.xticks(rotation=90, ha='right')
plt.show()

It looks like the confusion matrix has identified some common areas of confusion for our model. The model frequently mistakes white for for jeffrey pine, california red fir for white fir and jeffrey pine, jeffrey pine for white fir, and ponderosa pine for jeffrey pine.

What can we do to our model to improve predictions? Are there additional variables that you can think of that would help with the prediction?

In [ ]:
# User our new model to predict the SPCD of the trees detected in the ALS data
lidar_trees["predicted_spcd"] = model_spcd.predict(lidar_trees[independent_variables])
lidar_trees['predicted_species_name'] = lidar_trees['predicted_spcd'].apply(lambda x: spcd_to_common_name.get(x, f"Unknown ({x})"))

# Create a histogram of the predicted species
fig, ax = plt.subplots()
lidar_trees["predicted_species_name"].value_counts().plot(kind='bar', ax=ax)
ax.set_xlabel("Species")
ax.set_ylabel("Count")
ax.set_title("Independence Lake Tree Species Prediction")
plt.show()


### Crown Base Height (CBH) from Height

Crown base height (CBH), sometimes called live crown base height, is a measurment of how far above the ground the crown of the tree is. You can think of this as if you stood under a tree and stretched and stretched until your fingertips touch leaves or needles. How far you have to stretch (including your Go-Go-Gadget arm extenders) is the crown base height. 

Like species code, this is an important measurement because it tells us things like how likely a fire is to move from the surface into the tree, or how much foliage is in the crown. However, also like species code, we can't learn this from the ALS data and it is a difficult thing to predict. But, let's start simple and train a random forest model to predict crown base height just from the height of the tree.

In [ ]:
# Split into training and test sets
trees_fia["CBH"] = trees_fia["HT"] * (1 - trees_fia["CR"] / 100)
independent_variables = ["HT"]
dependent_variable = "CBH"
include_variables = independent_variables + [dependent_variable]
trees_train, trees_test = train_test_split(trees_fia[include_variables].dropna(), test_size=0.1)
print(f"Training set size: {len(trees_train)}")
print(f"Test set size: {len(trees_test)}")

# Train the model
model_cbh = RandomForestRegressor()
model_cbh.fit(trees_train[independent_variables], trees_train[dependent_variable])

# Evaluate model performance
print()
print(f"Model R^2: {model_cbh.score(trees_test[independent_variables], trees_test[dependent_variable]):.2f}")
print(f"Model RMSE: {((model_cbh.predict(trees_test[independent_variables]) - trees_test[dependent_variable])**2).mean()**0.5:.2f} feet")

# Compare the predicted CBH to the actual CBH.
fig, ax = plt.subplots()
trees_test["predicted_cbh"] = model_cbh.predict(trees_test[independent_variables])
trees_test.plot.scatter(x="CBH", y="predicted_cbh", ax=ax)
upper_cbh_limit = max(trees_test["CBH"].max(), trees_test["predicted_cbh"].max()) + 1
ax.plot([0, upper_cbh_limit], [0, upper_cbh_limit], color='red')
ax.set_xlabel("Actual CBH (ft)")
ax.set_ylabel("Predicted CBH (ft)")
plt.show()

While the R^2 suggests a reasonable fit to our model, the high RMSE suggests that there is a lot of work to be done on this metric. Unlike the relationship between height and diameter, which is often modeled as a simple exponential relationship, we tend to think of crown base height as more complicated. Live crown base height is often impacted by things like light availability, neighboring trees, and other landscape characteristics. Maybe there are other things from the FIA database, or from external data sources, that we can incorporate into the model.

In [ ]:
# User our new model to predict the CBH of the trees detected in the ALS data
lidar_trees["predicted_cbh"] = model_cbh.predict(lidar_trees[independent_variables])

# Plot the predicted CBH
fig, ax = plt.subplots()
lidar_trees.plot.scatter(x="HT", y="predicted_cbh", ax=ax)
ax.set_xlabel("Height (ft)")
ax.set_ylabel("Predicted CBH (ft)")
ax.set_title("Independence Lake Tree CBH Prediction")

# Plot sampled trees on the map with our new predicted values

In [ ]:
# Add the predicted diameter to the folium map
trees_sampled = lidar_trees.sample(10000)

# Get unique species names
species_names = sorted(trees_sampled['predicted_species_name'].unique())

# Create a color map based on unique species names using tab20
tab20 = plt.colormaps['tab20']
color_dict = {species_name: mcolors.to_hex(tab20(i % 20)) for i, species_name in enumerate(species_names)}

# Create the map
fmap = folium.Map(location=[trees_sampled.y.mean(), trees_sampled.x.mean()], zoom_start=15)

# Add the tile layer
tile = folium.TileLayer(
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='esri').add_to(fmap)

# Add markers for each tree
for idx, row in trees_sampled.iterrows():
    folium.CircleMarker(
        location=(row["y"], row["x"]),
        radius=3,
        weight=2,
        color=color_dict[row['predicted_species_name']],
        fill=True,
        fillColor=color_dict[row['predicted_species_name']],
        fillOpacity=0.7,
        popup=f"<b>Height:</b> {row.HT:.2f}ft<br><b>Diameter:</b> {row.predicted_diameter:.2f}in<br><b>Species Code:</b> {row.predicted_species_name}",
    ).add_to(fmap)

# Display the map
fmap

# Validation



Next, let's use our model to make predictions on a set of validation data. Field crews made around 300 tree observations and measured the quantities that we are interested in predicting: diameter, species code, and crown base height.

We'll take 25% of that data as our "validation" data, and the remaining 75% of the data will be used as "evalutation" data to evaluate student submissions.


In [ ]:
# Load the validation data
validation_data = pd.read_csv(DATA_PATH / 'validation_data.csv')

### Data cleaning

In [ ]:
# Rename tree_ht to HT
validation_data.rename(columns={'tree_ht': 'HT'}, inplace=True)

# Convert HT from meters to feet
validation_data["HT"] *= 3.28084

# Rename tree_dbh to DIA
validation_data.rename(columns={'tree_dbh': 'DIA'}, inplace=True)

# Convert DIA from centimeters to inches
validation_data["DIA"] *= 0.393701

# Rename tree_htlcb to CBH
validation_data.rename(columns={'tree_htlcb': 'CBH'}, inplace=True)

# Convert CBH from meters to feet
validation_data["CBH"] *= 3.28084

# Combine the Genus and Species columns in REF_SPECIES and use the tree_sp_scientific_name column to find the SPCD
fia_ref_species_table['SPECIES_NAME'] = fia_ref_species_table['GENUS'] + ' ' + fia_ref_species_table['SPECIES']
validation_data["SPCD"] = validation_data["tree_sp_scientific_name"].apply(lambda x: fia_ref_species_table[fia_ref_species_table['SPECIES_NAME'] == x]['SPCD'].values[0])

# Drop all columns except HT, DIA, CBH, and SPCD
validation_data = validation_data[["HT", "DIA", "CBH", "SPCD"]]

### Predictions

In [ ]:
# Predict the diameter of the validation data
validation_data["predicted_diameter"] = model_ht.predict(validation_data[["HT"]])

# Compute RMSE for the diameter prediction
dia_rmse = ((validation_data["DIA"] - validation_data["predicted_diameter"])**2).mean()**0.5
print(f"Diameter RMSE: {dia_rmse:.2f} inches")

# Compare the predicted diameter to the actual diameter.
fig, ax = plt.subplots()
validation_data.plot.scatter(x="DIA", y="predicted_diameter", ax=ax)
upper_dia_limit = max(validation_data["DIA"].max(), validation_data["predicted_diameter"].max()) + 1
ax.plot([0, upper_dia_limit], [0, upper_dia_limit], color='red')
ax.set_xlabel("Actual Diameter (in)")
ax.set_ylabel("Predicted Diameter (in)")
ax.set_title("Validation Data Diameter Prediction")
plt.show()

In [ ]:
# Predict the species of the validation data
validation_data["predicted_spcd"] = model_spcd.predict(validation_data[["HT"]])

# Compute the classification accuracy
report = classification_report(validation_data["SPCD"], validation_data["predicted_spcd"], zero_division=0)
print(report)

In [ ]:
# Predict the CBH of the validation data
validation_data["predicted_cbh"] = model_cbh.predict(validation_data[["HT"]])

# Compute RMSE for the CBH prediction
cbh_rmse = ((validation_data["CBH"] - validation_data["predicted_cbh"])**2).mean()**0.5
print(f"CBH RMSE: {cbh_rmse:.2f} feet")

# Compare the predicted CBH to the actual CBH.
fig, ax = plt.subplots()
validation_data.plot.scatter(x="CBH", y="predicted_cbh", ax=ax)
upper_cbh_limit = max(validation_data["CBH"].max(), validation_data["predicted_cbh"].max()) + 1
ax.plot([0, upper_cbh_limit], [0, upper_cbh_limit], color='red')
ax.set_xlabel("Actual CBH (ft)")
ax.set_ylabel("Predicted CBH (ft)")
ax.set_title("Validation Data CBH Prediction")
plt.show()

# Evaluation

For final evaluation of entries to the data challenge we will use the remaining 75% of the field data collected around Independence Lake to assess model accuracy. For this assesment, students are expected to provide a "submission.csv" file with the following additional columns:

- DIA
- SPCD
- CBH

In order to assess model accuracy we will compute the following metrics between your predictions and the actual observed values:

- RMSE for DIA and CBH
- F1 score for SPCD


In [ ]:
# Load the evaluation data
evaluation_data = pd.read_csv(DATA_PATH / 'evaluation_data.csv')

# Rename tree_ht to HT
evaluation_data.rename(columns={'tree_ht': 'HT'}, inplace=True)

# Convert HT from meters to feet
evaluation_data["HT"] *= 3.28084

# Predict the diameter of the evaluation data using the HT model
evaluation_data["DIA"] = model_ht.predict(evaluation_data[["HT"]])

# Predict the species of the evaluation data using the HT model
evaluation_data["SPCD"] = model_spcd.predict(evaluation_data[["HT"]])

# Predict the CBH of the evaluation data using the HT model
evaluation_data["CBH"] = model_cbh.predict(evaluation_data[["HT"]])

# Save the evaluation data to a CSV file named submission.csv
evaluation_data.to_csv("submission.csv", index=False)

In [ ]:
evaluation_data[]